## Data Loading and Preprocessing
This section loads and prepares the data for analysis.

In [ ]:
# Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Base directory for simulation evaluations
BASE_SIM_DIR = "sim_evals/"

# Predefined task and robot names
TASKS = ["GoToPosition", "GoToPose", "GoThroughPositions", "TrackVelocities"]
ROBOTS = ["FloatingPlatform", "Kingfisher", "Turtlebot2"]

# Color scheme for robots (Paired palette)
ROBOT_COLORS = {
    "FloatingPlatform": sns.color_palette("Paired")[5],  # Red
    "Kingfisher": sns.color_palette("Paired")[1],        # Blue
    "Turtlebot2": sns.color_palette("Paired")[3],        # Green
}

# Success thresholds
THRESHOLDS = {
    "GoToPosition": {"epsilon_p": 0.1},
    "GoToPose": {"epsilon_p": 0.1, "epsilon_theta": np.deg2rad(10)},
    "GoThroughPositions": {"min_goals_required": 5, "epsilon_p": 0.2},
    "TrackVelocities": {"tau_v": 0.2, "tau_w": np.deg2rad(10)},
}


## Defining Helper Functions
Here, we define utility functions to assist in the analysis.

In [ ]:
# Helper function to compute the moving average of a data series
def moving_average(data, window_size):
    return np.convolve(data, np.ones(window_size) / window_size, mode='valid')

In [ ]:
import pandas as pd
import numpy as np
import os
from glob import glob

def load_simulation_metrics(base_dir, task):
    """
    Load and organize simulation CSV results from the flattened format.

    Args:
        base_dir (str): Path to the 'sim_evals/' folder.
        task (str): Task name ('GoToPosition', 'GoToPose', 'GoThroughPositions').

    Returns:
        dict: Dictionary with robot names as keys and dicts of metric arrays as values.
    """
    task_dir = os.path.join(base_dir, task)
    csv_files = glob(os.path.join(task_dir, "*.csv"))

    sim_results = {}

    for csv_file in csv_files:
        robot_name = os.path.basename(csv_file).replace(".csv", "")
        df = pd.read_csv(csv_file)

        timesteps = int(df["timestep"].max()) + 1
        num_envs = int(df["env_id"].nunique())

        robot_data = {}

        for metric in df["metric"].unique():
            metric_df = df[df["metric"] == metric].copy()
            metric_pivot = metric_df.pivot(index="timestep", columns="env_id", values="value")
            metric_array = metric_pivot.values  # shape (timesteps, num_envs)
            robot_data[metric] = metric_array

        sim_results[robot_name] = robot_data

        print(f"✅ Loaded {robot_name} ({task}): {timesteps} timesteps, {num_envs} envs.")

    return sim_results


## Main Analysis Execution
This section contains the main computational steps for the analysis.

In [ ]:
def plot_simulation_results(sim_results, task_name, thresholds, split_axes=False):
    """
    Plot mean ± std curves for all robots for a given task and display a success rate table.
    """
    sns.set_theme(style="whitegrid")
    success_rates = {}
    heading_color = sns.color_palette("Paired")[7]  # Orange

    if task_name == "TrackVelocities":
        if split_axes:
            # === Separate Linear Velocity Error Plot ===
            plt.figure(figsize=(10, 6))
            for robot, metrics in sim_results.items():
                color = ROBOT_COLORS.get(robot, "black")
                lin_vel_errors = metrics["linear_velocity_errors"]

                mean_lin_vel = np.mean(lin_vel_errors, axis=1)
                std_lin_vel = np.std(lin_vel_errors, axis=1)
                timesteps = np.arange(len(mean_lin_vel))

                plt.plot(timesteps, mean_lin_vel, label=f"{robot}", color=color, linewidth=2)
                plt.fill_between(timesteps, mean_lin_vel - std_lin_vel, mean_lin_vel + std_lin_vel, alpha=0.2, color=color)

            plt.axhline(thresholds[task_name]["tau_v"], linestyle="--", color="black", label="Linear Velocity Threshold", linewidth=1.5)
            plt.xlabel("Timesteps", fontsize=12)
            plt.ylabel("Linear Velocity Error (m/s)", fontsize=12)
            plt.title(f"{task_name}: Linear Velocity Tracking", fontsize=14, fontweight="bold")
            plt.grid(True, linestyle="--", linewidth=0.5, alpha=0.4)
            plt.legend(loc="upper right", fontsize=10, frameon=True)
            plt.tight_layout()
            plt.show()

            # === Separate Angular Velocity Error Plot ===
            plt.figure(figsize=(10, 6))
            for robot, metrics in sim_results.items():
                color = ROBOT_COLORS.get(robot, "black")
                ang_vel_errors = metrics["angular_velocity_errors"]

                mean_ang_vel = np.mean(ang_vel_errors, axis=1)
                std_ang_vel = np.std(ang_vel_errors, axis=1)
                timesteps = np.arange(len(mean_ang_vel))

                plt.plot(timesteps, mean_ang_vel, label=f"{robot}", color=color, linewidth=2, linestyle="--")
                plt.fill_between(timesteps, mean_ang_vel - std_ang_vel, mean_ang_vel + std_ang_vel, alpha=0.2, color=color)

            plt.axhline(thresholds[task_name]["tau_w"], linestyle="--", color="orange", label="Angular Velocity Threshold", linewidth=1.5)
            plt.xlabel("Timesteps", fontsize=12)
            plt.ylabel("Angular Velocity Error (rad/s)", fontsize=12)
            plt.title(f"{task_name}: Angular Velocity Tracking", fontsize=14, fontweight="bold")
            plt.grid(True, linestyle="--", linewidth=0.5, alpha=0.4)
            plt.legend(loc="upper right", fontsize=10, frameon=True)
            plt.tight_layout()
            plt.show()

        else:
            # === Combined Linear + Angular Velocity Tracking ===
            fig, ax1 = plt.subplots(figsize=(10, 6))
            ax2 = ax1.twinx()

            for robot, metrics in sim_results.items():
                color = ROBOT_COLORS.get(robot, "black")
                lin_vel_errors = metrics["linear_velocity_errors"]
                ang_vel_errors = metrics["angular_velocity_errors"]

                mean_lin_vel = np.mean(lin_vel_errors, axis=1)
                std_lin_vel = np.std(lin_vel_errors, axis=1)
                mean_ang_vel = np.mean(ang_vel_errors, axis=1)
                std_ang_vel = np.std(ang_vel_errors, axis=1)
                timesteps = np.arange(len(mean_lin_vel))

                ax1.plot(timesteps, mean_lin_vel, label=f"{robot} - Linear Velocity", color=color, linewidth=2)
                ax1.fill_between(timesteps, mean_lin_vel - std_lin_vel, mean_lin_vel + std_lin_vel, alpha=0.2, color=color)

                ax2.plot(timesteps, mean_ang_vel, label=f"{robot} - Angular Velocity", color=color, linestyle="--", linewidth=2)
                ax2.fill_between(timesteps, mean_ang_vel - std_ang_vel, mean_ang_vel + std_ang_vel, alpha=0.2, color=color)

            ax1.axhline(thresholds[task_name]["tau_v"], linestyle="--", color="black", label="Linear Velocity Threshold", linewidth=1.5)
            ax2.axhline(thresholds[task_name]["tau_w"], linestyle="--", color="orange", label="Angular Velocity Threshold", linewidth=1.5)

            ax1.set_xlabel("Timesteps", fontsize=12)
            ax1.set_ylabel("Linear Velocity Error (m/s)", fontsize=12)
            ax2.set_ylabel("Angular Velocity Error (rad/s)", fontsize=12, color=heading_color)

            ax2.tick_params(axis='y', labelcolor=heading_color)
            
            ax1.set_ylim(bottom=0)
            ax2.set_ylim(bottom=0)
            ax1.grid(True, linestyle="--", linewidth=0.5, alpha=0.4)
            ax2.grid(False)
            handles1, labels1 = ax1.get_legend_handles_labels()
            handles2, labels2 = ax2.get_legend_handles_labels()
            plt.legend(handles1 + handles2, labels1 + labels2, loc="upper right", fontsize=10, frameon=True)

            plt.title(f"{task_name}: Simulation Performance", fontsize=14, fontweight="bold")
            plt.tight_layout()
            plt.show()
        
        # tracking if mean velocity is within threshold for each robot
        for robot, metrics in sim_results.items():

            lin_vel_errors = metrics["linear_velocity_errors"]
            ang_vel_errors = metrics["angular_velocity_errors"]

            mean_lin_vel = np.mean(lin_vel_errors, axis=1)
            mean_ang_vel = np.mean(ang_vel_errors, axis=1)

            success = np.mean((np.abs(mean_lin_vel) < thresholds[task_name]["tau_v"]) & (np.abs(mean_ang_vel) < thresholds[task_name]["tau_w"]))
            success_rates[robot] = f"{success*100:.1f}%"
        

    elif task_name == "GoToPose":
        success = {}

        if split_axes:
            # === Separate Distance Plot ===
            plt.figure(figsize=(10, 6))
            for robot, metrics in sim_results.items():
                color = ROBOT_COLORS.get(robot, "black")
                distances = metrics["distance_to_goal"]
                mean_distance = np.mean(distances, axis=1)
                std_distance = np.std(distances, axis=1)
                timesteps = np.arange(len(mean_distance))

                plt.plot(timesteps, mean_distance, label=f"{robot}", color=color, linewidth=2)
                plt.fill_between(timesteps, mean_distance - std_distance, mean_distance + std_distance, alpha=0.2, color=color)

            plt.axhline(thresholds[task_name]["epsilon_p"], linestyle="--", color="black", label="Distance Threshold (0.1m)", linewidth=1.5)
            plt.xlabel("Timesteps", fontsize=12)
            plt.ylabel("Distance to Goal (m)", fontsize=12)
            plt.title(f"{task_name}: Distance Convergence", fontsize=14, fontweight="bold")
            plt.grid(True, linestyle="--", linewidth=0.5, alpha=0.4)
            plt.legend(loc="upper right", fontsize=10, frameon=True)
            plt.tight_layout()
            plt.show()

            # === Separate Heading Plot ===
            plt.figure(figsize=(10, 6))
            for robot, metrics in sim_results.items():
                color = ROBOT_COLORS.get(robot, "black")
                headings = np.rad2deg(metrics["heading_errors"])
                mean_heading = np.mean(headings, axis=1)
                std_heading = np.std(headings, axis=1)
                timesteps = np.arange(len(mean_heading))

                plt.plot(timesteps, mean_heading, label=f"{robot}", color=color, linewidth=2, linestyle="--")
                plt.fill_between(timesteps, mean_heading - std_heading, mean_heading + std_heading, alpha=0.2, color=color)

            plt.axhline(np.rad2deg(thresholds[task_name]["epsilon_theta"]), linestyle="--", color="orange", label="Heading Threshold (10°)", linewidth=1.5)
            plt.xlabel("Timesteps", fontsize=12)
            plt.ylabel("Heading Error (°)", fontsize=12)
            plt.title(f"{task_name}: Heading Alignment", fontsize=14, fontweight="bold")
            plt.grid(True, linestyle="--", linewidth=0.5, alpha=0.4)
            plt.legend(loc="upper right", fontsize=10, frameon=True)
            plt.tight_layout()
            plt.show()

        else:
            # === Combined Distance + Heading Dual-Axis Plot ===
            fig, ax1 = plt.subplots(figsize=(10, 6))
            ax2 = ax1.twinx()

            for robot, metrics in sim_results.items():
                color = ROBOT_COLORS.get(robot, "black")

                distances = metrics["distance_to_goal"]
                headings = metrics["heading_errors"]

                mean_distance = np.mean(distances, axis=1)
                std_distance = np.std(distances, axis=1)
                mean_heading = np.rad2deg(np.mean(headings, axis=1))
                std_heading = np.rad2deg(np.std(headings, axis=1))
                timesteps = np.arange(len(mean_distance))

                ax1.plot(timesteps, mean_distance, label=f"{robot} - Distance", color=color, linewidth=2)
                ax1.fill_between(timesteps, mean_distance - std_distance, mean_distance + std_distance, alpha=0.2, color=color)

                ax2.plot(timesteps, mean_heading, label=f"{robot} - Heading", color=color, linestyle="--", linewidth=2)
                ax2.fill_between(timesteps, mean_heading - std_heading, mean_heading + std_heading, alpha=0.2, color=color)

            ax1.axhline(thresholds[task_name]["epsilon_p"], linestyle="--", color="red", label="Distance Threshold (0.1m)", linewidth=1.5)
            ax2.axhline(np.rad2deg(thresholds[task_name]["epsilon_theta"]), linestyle="--", color="orange", label="Heading Threshold (10°)", linewidth=1.5)

            ax1.set_xlabel("Timesteps", fontsize=12)
            ax1.set_ylabel("Distance to Goal (m)", fontsize=12)
            ax2.set_ylabel("Heading Error (°)", fontsize=12, color="orange")
            ax1.set_ylim(bottom=0)
            ax2.set_ylim(bottom=0)
            ax1.grid(True, linestyle="--", linewidth=0.5, alpha=0.4)

            handles1, labels1 = ax1.get_legend_handles_labels()
            handles2, labels2 = ax2.get_legend_handles_labels()
            plt.legend(handles1 + handles2, labels1 + labels2, loc="upper right", fontsize=10, frameon=True)

            plt.title(f"{task_name}: Simulation Performance", fontsize=14, fontweight="bold")
            plt.tight_layout()
            plt.show()
        
        for robot, metrics in sim_results.items():
            distances = metrics["distance_to_goal"]
            headings = metrics["heading_errors"]

            mean_distance = np.mean(distances, axis=1)
            mean_heading = np.rad2deg(np.mean(headings, axis=1))
            # save success for distance and heading separately for each robot
            success_distance = np.mean(mean_distance < thresholds[task_name]["epsilon_p"])
            success_heading = np.mean(mean_heading < thresholds[task_name]["epsilon_theta"])
            success_rates[robot] = f"{success_distance*100:.1f}% / {success_heading*100:.1f}%"
        
            
    elif task_name == "GoThroughPositions":
        # Cumulative Goals Plot
        plt.figure(figsize=(10, 6))
        min_goals_required = thresholds[task_name]["min_goals_required"]
        
        for robot, metrics in sim_results.items():
            color = ROBOT_COLORS.get(robot, "black")
            cumulative_goals = metrics["cumulative_goals"]
            timesteps = np.arange(cumulative_goals.shape[0])

            mean_goals = np.mean(cumulative_goals, axis=1)
            std_goals = np.std(cumulative_goals, axis=1)

            # Optional smoothing
            window_size = 20
            smooth_mean_goals = moving_average(mean_goals, window_size)
            smooth_std_goals = moving_average(std_goals, window_size)
            smooth_timesteps = timesteps[:len(smooth_mean_goals)]

            plt.plot(smooth_timesteps, smooth_mean_goals, label=f"{robot}", color=color, linewidth=2)
            plt.fill_between(
                smooth_timesteps,
                smooth_mean_goals - smooth_std_goals,
                smooth_mean_goals + smooth_std_goals,
                alpha=0.2,
                color=color
            )

            # Success rate: percentage of envs with at least `min_goals_required`
            final_goals_per_env = cumulative_goals[-1, :]
            success = np.mean(final_goals_per_env >= min_goals_required)
            success_rates[robot] = f"{success * 100:.1f}%"

        plt.xlabel("Timesteps", fontsize=12)
        plt.ylabel("Cumulative Goals Reached", fontsize=12)
        plt.title(f"{task_name}: Cumulative Goals Over Time", fontsize=14, fontweight="bold")
        plt.grid(True, linestyle="--", linewidth=0.5, alpha=0.4)
        plt.legend(loc="upper left", fontsize=10, frameon=True)
        plt.tight_layout()
        plt.show()


        TOTAL_ENVS = 4096  
        plt.figure(figsize=(10, 6))

        for robot, metrics in sim_results.items():
            color = ROBOT_COLORS.get(robot, "black")
            final_goals = metrics["cumulative_goals"][-1, :]

            sns.histplot(
                final_goals,
                bins=range(int(final_goals.min()), int(final_goals.max()) + 1),  # Keep integer bins
                kde=True,  
                kde_kws={"bw_adjust": 2.5},  # Adjust KDE to avoid multiple peaks
                stat="percent",  # Converts the histogram into percentage
                color=color,
                label=f"{robot}",
                alpha=0.5,
                edgecolor="black",
                linewidth=0.6
            )

        # Customize Y-axis to explicitly show percentages
        plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f"{y:.1f}%"))

        # Enhance plot aesthetics
        plt.xlabel("Total Goals Reached", fontsize=12)
        plt.ylabel("Percentage of Environments", fontsize=12)
        plt.title(f"{task_name}: Distribution of Goals Reached per Environment", fontsize=14, fontweight="bold")
        plt.grid(True, linestyle="--", linewidth=0.5, alpha=0.4)
        plt.legend(loc="upper right", fontsize=10, frameon=True)
        plt.tight_layout()

        plt.show()

    else:
        plt.figure(figsize=(10, 6))

        for robot, metrics in sim_results.items():
            color = ROBOT_COLORS.get(robot, "black")

            if task_name == "GoToPosition":
                data = metrics["distance_to_goal"]
                mean_distance = np.mean(data, axis=1)
                std_distance = np.std(data, axis=1)
                timesteps = np.arange(len(mean_distance))

                plt.plot(timesteps, mean_distance, label=f"{robot}", color=color, linewidth=2)
                plt.fill_between(timesteps, mean_distance - std_distance, mean_distance + std_distance, alpha=0.2, color=color)

                final_distances = data[-1, :]
                success = np.mean(final_distances < thresholds[task_name]["epsilon_p"])
                success_rates[robot] = f"{success*100:.1f}%"

            elif task_name == "GoThroughPositions":
                success_rate = metrics["waypoint_success_rate"]
                mean_success = np.mean(success_rate, axis=1)
                std_success = np.std(success_rate, axis=1)
                timesteps = np.arange(len(mean_success))

                plt.plot(timesteps, mean_success, label=f"{robot}", color=color, linewidth=2)
                plt.fill_between(timesteps, mean_success - std_success, mean_success + std_success, alpha=0.2, color=color)

                final_success = np.mean(success_rate[-1, :])
                success_rates[robot] = f"{final_success*100:.1f}%"

        if task_name == "GoToPosition":
            plt.axhline(thresholds["GoToPosition"]["epsilon_p"], linestyle="--", color="black", label="Threshold (0.1m)", linewidth=1.5)

        plt.xlabel("Timesteps", fontsize=12)
        ylabel = {
            "GoToPosition": "Distance to Goal (m)",
            "GoThroughPositions": "Success Rate"
        }
        plt.ylabel(ylabel[task_name], fontsize=12)
        plt.title(f"{task_name}: Simulation Performance", fontsize=14, fontweight="bold")
        plt.grid(True, linestyle="--", linewidth=0.5, alpha=0.4)
        plt.legend(loc="upper right", fontsize=10, frameon=True)
        plt.tight_layout()
        plt.show()
        
    # Success rate table
    success_table = pd.DataFrame.from_dict(success_rates, orient="index", columns=["Success Rate"])
    success_table.index.name = "Robot"
    success_table["Task"] = task_name
    display(success_table)


## Visualization and Results
We generate plots and summary statistics.

In [ ]:
BASE_SIM_DIR = "../../../../../sim_evals/"
TASK = "GoThroughPositions"
sim_results = load_simulation_metrics(BASE_SIM_DIR, TASK)
plot_simulation_results(sim_results, TASK, THRESHOLDS, split_axes=True)



## Helper functions for Field Tests Experiments
load_field_tests: Loads and selects top-N field test runs for a robot-task pair. 

aggregate_field_runs: Aggregate top-N runs into mean and std arrays over time.


In [ ]:
from glob import glob

def load_field_tests(base_dir, robot, task, top_n=5, metric="distance_error.m"):
    """
    Load and select top-N field test runs for a robot-task pair.

    Args:
        base_dir (str): Path to 'field_tests_data/'.
        robot (str): Robot name.
        task (str): Task name.
        top_n (int): Number of best runs to select.
        metric (str): Metric to sort runs by (e.g., "distance_error.m").

    Returns:
        list: List of top-N DataFrames (one per run).
    """
    folder_path = os.path.join(base_dir, robot, task)
    csv_files = glob(os.path.join(folder_path, "*.csv"))

    runs = []
    scores = []

    for file in csv_files:
        df = pd.read_csv(file)
        # save runs name
        df['run_id'] = file.split('/')[-1].split('.')[0]
        if metric in df.columns:
            final_value = df[metric].iloc[-1]
            runs.append(df)
            scores.append(final_value)

    if not runs:
        print(f"⚠️ No valid runs found for {robot} - {task}.")
        return []

    # Select top-N based on lowest final metric value
    top_indices = np.argsort(scores)[:top_n]
    top_runs = [runs[i] for i in top_indices]

    print(f"✅ Loaded {len(top_runs)} top runs for {robot} - {task}.")
    return top_runs

In [ ]:
def aggregate_field_runs(runs, metric):
    """
    Aggregate top-N runs into mean and std arrays over time.

    Args:
        runs (list): List of DataFrames.
        metric (str): Metric to extract.

    Returns:
        (np.ndarray, np.ndarray): Mean and std arrays over time.
    """
    all_runs = [run[metric].values for run in runs]
    min_length = min(len(run) for run in all_runs)

    # Trim runs to the shortest length to align
    trimmed_runs = [run[:min_length] for run in all_runs]
    data = np.vstack(trimmed_runs)

    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)

    return mean, std


In [ ]:
def compute_field_success_rate(runs, task, thresholds):
    """
    Compute success rate over top-N runs.
    """
    success_count = 0
    for run in runs:
        if task == "GoToPosition":
            if run["distance_error.m"].iloc[-1] < thresholds[task]["epsilon_p"]:
                success_count += 1
        elif task == "GoToPose":
            if (run["distance_error.m"].iloc[-1] < thresholds[task]["epsilon_p"] and
                abs(run["heading_error.rad"].iloc[-1]) < thresholds[task]["epsilon_theta"]):
                success_count += 1
        elif task == "GoThroughPositions":
            if run["num_goals_reached.u"].max() > 0:
                success_count += 1
    print(runs[0]["distance_error.m"])

    return f"{(success_count / len(runs)) * 100:.1f}%"

In [ ]:
def compute_cumulative_goals_field(run, threshold=0.2, robot="Kingfisher"):
    """
    Compute cumulative goals reached for a single field test run, handling column swaps for specific robots.

    Args:
        run (pd.DataFrame): Field test run dataframe.
        robot (str): Name of the robot (used for column correction).
        threshold (float): Distance threshold to consider a goal as reached.

    Returns:
        np.ndarray: Cumulative goals reached over time.
    """
    # Detect the correct column based on the robot type
    # if robot == "Kingfisher":
    #     distance_column = "linear_velocities_bodyx.m/s"  # Fix the swap
    # else:
    distance_column = "task_data.dist.m"

    # Ensure the column exists before proceeding
    if distance_column not in run.columns:
        raise ValueError(f"Column '{distance_column}' not found in the field test data for {robot}")

    # simply return num_goals_reached
    # return run["num_goals_reached.u"].values
    distance_column = "task_data.dist.m"
    distances = run[distance_column].values

    goals_reached = run["num_goals_reached.u"].values
    tot_goals = 0
    cumulative_goals = np.zeros_like(distances)

    for t in range(len(distances)):
        #if turtlebot2, we need to add 1 to the total goals reached
        if robot == "Turtlebot2":
            goals_reached[t] += 1
        if distances[0] < threshold and goals_reached[t] > tot_goals:
            tot_goals = goals_reached[t]  # update total goals reached
        cumulative_goals[t] = tot_goals

    return cumulative_goals

    # for t in range(len(distances)):
    #     if distances[t] < threshold and not goal_reached_flag:
    #         total_goals += 1
    #         goal_reached_flag = True
    #     elif distances[t] > threshold:
    #         goal_reached_flag = False
    #     cumulative_goals[t] = total_goals

    # return cumulative_goals


## Plot function for the field test, following the same scheme for simulation results
We generate plots and summary statistics.

In [ ]:
def plot_field_test_results(top_runs, task, thresholds, robot, save_path=None):
    """
    Plot mean ± std curves for field test top-N runs with success rate and twin axes for GoToPose.
    """
    sns.set_theme(style="whitegrid")
    color_distance = ROBOT_COLORS.get(robot, "black")
    color_heading = sns.color_palette("Paired")[7]
    color_goals = sns.color_palette("Paired")[2]
    color_goals_fill = sns.color_palette("Paired")[3]
    threshold_distance = "black"
    threshold_heading = "orange"

    timesteps = np.arange(min(len(run) for run in top_runs))
    print(f"[INFO] Run lengths for {robot} - {task}: {[len(run) for run in top_runs]}")

    if task == "GoToPosition":
        mean_distance, std_distance = aggregate_field_runs(top_runs, "distance_error.m")

        fig, ax = plt.subplots(figsize=(10, 6))
        ax.set_xlabel("Timesteps")
        ax.set_ylabel("Distance to Goal (m)", color=color_distance)
        ax.plot(timesteps, mean_distance, label="Mean Distance Error", color=color_distance, linewidth=2)
        ax.fill_between(timesteps, mean_distance - std_distance, mean_distance + std_distance, alpha=0.2, color=color_distance)
        ax.axhline(thresholds[task]["epsilon_p"], linestyle="--", color=threshold_distance, label="Distance Threshold (0.1m)", linewidth=1.5)
        ax.tick_params(axis='y', labelcolor=color_distance)
        ax.set_ylim(bottom=0)
        ax.grid(True, linestyle="--", linewidth=0.5, alpha=0.4)
        plt.title(f"{robot} - {task} Field Test Performance", fontsize=14, fontweight="bold")
        fig.tight_layout()
        ax.legend(loc="upper right", fontsize=10, frameon=True)
        if save_path:
            plt.savefig(os.path.join(save_path, f"{robot}_{task}_field_results.png"), dpi=200)
        plt.show()

    elif task == "GoToPose":
        mean_distance, std_distance = aggregate_field_runs(top_runs, "distance_error.m")
        mean_heading, std_heading = aggregate_field_runs(top_runs, "heading_error.rad")
        mean_heading = np.abs(np.rad2deg(mean_heading))
        std_heading = np.abs(np.rad2deg(std_heading))

        fig, ax1 = plt.subplots(figsize=(10, 6))
        ax1.set_xlabel("Timesteps")
        ax1.set_ylabel("Distance Error (m)", color=color_distance)
        ax1.plot(timesteps, mean_distance, label="Mean Distance Error", color=color_distance, linewidth=2)
        ax1.fill_between(timesteps, mean_distance - std_distance, mean_distance + std_distance, alpha=0.2, color=color_distance)
        ax1.axhline(thresholds[task]["epsilon_p"], linestyle="--", color=threshold_distance, label="Distance Threshold (0.1m)", linewidth=1.5)
        ax1.tick_params(axis='y', labelcolor=color_distance)

        ax2 = ax1.twinx()
        ax2.set_ylabel("Heading Error (°)", color=color_heading)
        ax2.plot(timesteps, mean_heading, label="Mean Heading Error", color=color_heading, linestyle="--", linewidth=2)
        ax2.fill_between(timesteps, mean_heading - std_heading, mean_heading + std_heading, alpha=0.2, color=color_heading)
        ax2.axhline(np.rad2deg(thresholds[task]["epsilon_theta"]), linestyle="--", color=threshold_heading, label="Heading Threshold (10°)", linewidth=1.5)
        ax2.tick_params(axis='y', labelcolor=color_heading)

        ax1.set_ylim(bottom=0)
        ax2.set_ylim(bottom=0)

        # Only enable grid for the primary y-axis (distance error)
        ax1.grid(True, linestyle="--", linewidth=0.5, alpha=0.4)
        ax2.grid(False)  # Disable grid for the secondary y-axis (heading error)

        plt.title(f"{robot} - {task} Field Test Performance", fontsize=14, fontweight="bold")
        fig.tight_layout()

        handles1, labels1 = ax1.get_legend_handles_labels()
        handles2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(handles1 + handles2, labels1 + labels2, loc="upper right", fontsize=10, frameon=True)

        if save_path:
            plt.savefig(os.path.join(save_path, f"{robot}_{task}_field_results.png"), dpi=200)
        plt.show()
        
    elif task == "GoThroughPositions":
        # Compute cumulative goals for all runs
        cumulative_goals_runs = []
        final_goals_per_run = []
        for run in top_runs:
            cumulative_goals = compute_cumulative_goals_field(run, threshold=thresholds[task]["epsilon_p"])
            cumulative_goals_runs.append(cumulative_goals)
            final_goals_per_run.append(cumulative_goals[-1])  # Take final goal count

        # Handle variable-length runs by trimming to the shortest
        min_length = min(len(cg) for cg in cumulative_goals_runs)
        cumulative_goals_runs = np.array([cg[:min_length] for cg in cumulative_goals_runs])
        timesteps = np.arange(min_length)

        # Aggregate mean and std
        mean_goals = np.mean(cumulative_goals_runs, axis=0)
        std_goals = np.std(cumulative_goals_runs, axis=0)

        # Optional smoothing
        window_size = 100
        smooth_mean_goals = moving_average(mean_goals, window_size)
        smooth_std_goals = moving_average(std_goals, window_size)
        smooth_timesteps = timesteps[:len(smooth_mean_goals)]

        # Plot cumulative goals over time
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.set_xlabel("Timesteps")
        ax.set_ylabel("Cumulative Goals Reached", fontsize=12)
        ax.plot(smooth_timesteps, smooth_mean_goals, label="Smoothed Mean Goals Reached", color=color_goals, linewidth=2)
        ax.fill_between(smooth_timesteps, smooth_mean_goals - smooth_std_goals, smooth_mean_goals + smooth_std_goals, alpha=0.2, color=color_goals_fill)
        ax.set_ylim(bottom=0)
        ax.grid(True, linestyle="--", linewidth=0.5, alpha=0.4)
        plt.title(f"{robot} - {task} Field Test Performance", fontsize=14, fontweight="bold")
        fig.tight_layout()
        ax.legend(loc="upper left", fontsize=10, frameon=True)

        if save_path:
            plt.savefig(os.path.join(save_path, f"{robot}_{task}_field_results_cumulative_goals.png"), dpi=200)
        plt.show()

        # Plot histogram with KDE overlay
        fig, ax = plt.subplots(figsize=(10, 6))
        # Histogram with density normalization for smoothness
        sns.histplot(final_goals_per_run, binwidth=1, color="blue", kde=False, stat="density", alpha=0.6)
        # KDE for smooth interpolation
        sns.kdeplot(final_goals_per_run, bw_adjust=0.5, color="red", linewidth=2, alpha=0.8)
        # Labels and customization
        ax.set_xlabel("Total Goals Reached", fontsize=12)
        ax.set_ylabel("Density", fontsize=12)
        ax.set_xlim(left=0)
        ax.set_xticks(range(int(max(final_goals_per_run)) + 1))
        ax.set_title(f"{robot} - {task}: Goals Reached Distribution", fontsize=14, fontweight="bold")
        ax.grid(True, linestyle="--", linewidth=0.5, alpha=0.4)


    # Save figure if needed
    if save_path:
        plt.savefig(os.path.join(save_path, f"{robot}_{task}_field_results_goals_histogram.png"), dpi=200)

    plt.show()



## Visualization and Summary Statistics

We allow the selection of task, robot pairs, to load all the runs present in the field_test experiment folders and plot the results.

The tests can be filtered based on the best top_n runs.

In [ ]:
BASE_FIELD_DIR = "../../../../../field_tests_data"

robot = "FloatingPlatform"
task = "GoToPose"

all_runs = load_field_tests(BASE_FIELD_DIR, robot, task, top_n=20)
plot_field_test_results(all_runs, task, THRESHOLDS, robot)
# Success rate
success_rate = compute_field_success_rate(all_runs, task, THRESHOLDS)
success_table = pd.DataFrame({robot: [success_rate]}, index=["Success Rate"])
display(success_table)

# Additional filtering and test comparisons

In the following experiments are filtered by length or specific performance thresholds to see the difference 

In [ ]:
long_runs = [run for run in all_runs if len(run) > 2000]
short_runs = [run for run in all_runs if len(run) <= 1000]
medium_runs = [run for run in all_runs if 1000 < len(run) <= 2000]

In [ ]:
# get bad runs and print their tot number and names
bad_runs = []
good_runs = []
for run in all_runs:
    if run["distance_error.m"].iloc[-1] > 2 * THRESHOLDS[task]["epsilon_p"]:
        bad_runs.append(run)
    else:
        good_runs.append(run)

print(f"Found {len(bad_runs)} bad runs for {robot} - {task} and {len(good_runs)} good runs.")


In [ ]:
# plot the good runs
plot_field_test_results(good_runs, task, THRESHOLDS, robot)
# Success rate
success_rate = compute_field_success_rate(good_runs, task, THRESHOLDS)
success_table = pd.DataFrame({robot: [success_rate]}, index=["Success Rate"])
display(success_table)

# Using effective experiment time (s) instead of timesteps

In [ ]:
def plot_gotoposition_comparison_timesteps(all_robot_runs, thresholds, save_path=None):
    """
    Plot mean ± std curves for GoToPosition task comparing multiple robots, using timesteps as the x-axis.

    Args:
        all_robot_runs (dict): Dictionary with robot names as keys and lists of runs (DataFrames) as values.
        thresholds (dict): Dictionary of success thresholds for GoToPosition.
        save_path (str, optional): Directory to save plots.
    """
    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(10, 6))

    for robot, runs in all_robot_runs.items():
        color = ROBOT_COLORS.get(robot, "black")
        distance_runs = []

        for run in runs:
            if isinstance(run, pd.DataFrame):
                if "distance_error.m" not in run.columns:
                    print(f"⚠️ Skipping {robot}: Missing required columns")
                    continue
                distance_runs.append(run["distance_error.m"].values)
            else:
                print(f"⚠️ Skipping {robot}: Run is not a DataFrame")
                continue

        if not distance_runs:
            print(f"⚠️ No valid runs for {robot}")
            continue

        # Find the minimum run length to align all trajectories
        min_length = min(len(run) for run in distance_runs)
        trimmed_runs = np.array([run[:min_length] for run in distance_runs])

        # Compute mean and std
        mean_distance = np.mean(trimmed_runs, axis=0)
        std_distance = np.std(trimmed_runs, axis=0)
        timesteps = np.arange(min_length)

        # Plot mean ± std
        plt.plot(timesteps, mean_distance, label=f"{robot}", color=color, linewidth=2)
        plt.fill_between(
            timesteps,
            mean_distance - std_distance,
            mean_distance + std_distance,
            alpha=0.2,
            color=color
        )

    # Threshold line
    plt.axhline(thresholds["GoToPosition"]["epsilon_p"], linestyle="--", color="black", label="Distance Threshold (0.1m)", linewidth=1.5)

    # Labels and aesthetics
    plt.xlabel("Timesteps", fontsize=12)
    plt.ylabel("Distance to Goal (m)", fontsize=12)
    plt.title("GoToPosition: Comparison Across Robots", fontsize=14, fontweight="bold")
    plt.grid(True, linestyle="--", linewidth=0.5, alpha=0.4)
    plt.legend(loc="upper right", fontsize=10, frameon=True)
    plt.tight_layout()

    # Save plot if requested
    if save_path:
        plt.savefig(os.path.join(save_path, "GoToPosition_comparison_timesteps.png"), dpi=200)

    plt.show()


In [ ]:
task_in_use = "GoToPosition"
all_robot_runs_gotoposition = {
    "FloatingPlatform": load_field_tests(BASE_FIELD_DIR, "FloatingPlatform", task_in_use, top_n=20),
    "Kingfisher": good_runs,
    "Turtlebot2": load_field_tests(BASE_FIELD_DIR, "Turtlebot2", task_in_use, top_n=20),
}


In [ ]:
plot_gotoposition_comparison_timesteps(all_robot_runs_gotoposition, THRESHOLDS)


In [ ]:
def filter_runs_by_length(all_runs, min_length, max_length):
    """
    Filters runs by their trajectory length.
    """
    return [run for run in all_runs if min_length <= len(run) <= max_length]


In [ ]:
# Length range for medium runs
MIN_LENGTH = 0
MAX_LENGTH = 1000
threshold = THRESHOLDS["GoThroughPositions"]["epsilon_p"]

robots = ["FloatingPlatform", "Kingfisher", "Turtlebot2"]
filtered_runs = {}

# Filter and compute cumulative goals
for robot in robots:
    all_runs = load_field_tests(BASE_FIELD_DIR, robot, "GoThroughPositions", top_n=20)
    runs_in_range = filter_runs_by_length(all_runs, MIN_LENGTH, MAX_LENGTH)
    print(f"✅ {robot}: {len(runs_in_range)} runs in range [{MIN_LENGTH}, {MAX_LENGTH}]")
    
    cumulative_goals_runs = []
    for run in runs_in_range:
        cumulative_goals = compute_cumulative_goals_field(run, threshold, robot)
        cumulative_goals_runs.append(cumulative_goals)
    
    filtered_runs[robot] = cumulative_goals_runs


In [ ]:
# Align lengths
valid_runs = {robot: runs for robot, runs in filtered_runs.items() if runs}  # Remove empty entries
if not valid_runs:
    raise ValueError("No valid runs in the selected length range.")

min_timesteps = min(min(len(run) for run in runs) for runs in valid_runs.values())
timesteps = np.arange(min_timesteps)

plt.figure(figsize=(10, 6))
for robot, runs in filtered_runs.items():
    color = ROBOT_COLORS.get(robot, "black")
    goals_matrix = np.array([run[:min_timesteps] for run in runs])
    
    mean_goals = np.mean(goals_matrix, axis=0)
    std_goals = np.std(goals_matrix, axis=0)
    
    window_size = 20
    smooth_mean = moving_average(mean_goals, window_size)
    smooth_std = moving_average(std_goals, window_size)
    smooth_timesteps = timesteps[:len(smooth_mean)]

    plt.plot(smooth_timesteps, smooth_mean, label=f"{robot}", color=color, linewidth=2)
    plt.fill_between(
        smooth_timesteps,
        smooth_mean - smooth_std,
        smooth_mean + smooth_std,
        alpha=0.2,
        color=color
    )

plt.xlabel("Timesteps", fontsize=12)
plt.ylabel("Cumulative Goals Reached", fontsize=12)
plt.title("GoThroughPositions: Field Tests Cumulative Goals", fontsize=14, fontweight="bold")
plt.grid(True, linestyle="--", linewidth=0.5, alpha=0.4)
plt.legend(loc="upper left", fontsize=10, frameon=True)
plt.tight_layout()
plt.show()


# Additional plot for TrackVelocity task

In [ ]:

def plot_track_velocity_trajectories(csv_folder, task="TrackVelocity", save_path=None):
    """
    Loads trajectory data from multiple CSV files and plots the target trajectory 
    along with actual robot trajectories for the TrackVelocity task.

    Args:
        csv_folder (str): Path to the folder containing CSV trajectory files.
        task (str): Task name, default is "TrackVelocity".
        save_path (str, optional): Path to save the figure. If None, the plot is displayed.
    """
    fig, ax = plt.subplots(figsize=(8, 8))  # Ensure a square figure
    ax.set_xlabel("X Position (m)")
    ax.set_ylabel("Y Position (m)")
    ax.set_title(f"{task} Field Test Trajectories", fontsize=14, fontweight="bold")

    # Load Seaborn Paired color palette
    colors = sns.color_palette("Paired", 12)  # Use 12 distinct colors

    # Load and plot each trajectory
    csv_files = sorted([f for f in os.listdir(csv_folder) if f.endswith(".csv")])
    for i, file in enumerate(csv_files):
        file_path = os.path.join(csv_folder, file)
        df = pd.read_csv(file_path)

        # Extract world position (X, Y) for actual trajectory
        x, y = df["position_world.x.m"], df["position_world.y.m"]

        # Assign colors and plot trajectory
        color = colors[i % len(colors)]  # Cycle colors if there are more than 12 files
        ax.plot(x, y, label="Actual trajectory", color=color, linewidth=2)
        ax.scatter(x.iloc[0], y.iloc[0], marker="o", color=color, edgecolors="black", s=80, label=None)  # Start point

        # Extract unique target positions (goal locations)
        goals = df[["target_position.x.m", "target_position.y.m"]].drop_duplicates()
        ax.scatter(goals["target_position.x.m"], goals["target_position.y.m"], 
                   color="black", marker="o", s=10, label="Goal trajectory" if i == 0 else None)  # Label only once

    # Ensure a balanced (square) aspect ratio
    ax.set_aspect('equal', adjustable='datalim')

    ax.legend(loc="best", fontsize=10, frameon=True)
    ax.grid(True, linestyle="--", linewidth=0.5, alpha=0.4)

    if save_path:
        plt.savefig(os.path.join(save_path, f"{task}_field_trajectories.png"), dpi=200)
    plt.show()


In [ ]:
csv_folder = BASE_FIELD_DIR + "/Kingfisher/TrackVelocities"
plot_track_velocity_trajectories(csv_folder)
